## Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np

## Importando Base de Dados

In [2]:
feData = pd.read_csv('data/ifood-restaurants-february-2021.csv')
noData = pd.read_csv('data/ifood-restaurants-november-2020.csv')

feData.head()

,availableForScheduling,avatar,category,delivery_fee,delivery_time,distance,ibge,minimumOrderValue,name,paymentCodes,price_range,rating,tags,url
0,False,https://static-images.ifood.com.br/image/uploa...,Marmita,3.99,27,1.22,5300108,10.0,Cantina Arte & Sabor,DNR $$ MPAY $$ MOVPAY_MC $$ MC $$ GPY_ELO $$ E...,CHEAPEST,0.0,ADDRESS_PREFORM_TYPE $$ CART::MCHT::100_DELIVE...,https://www.ifood.com.br/delivery/brasilia-df/...
1,False,https://static-images.ifood.com.br/image/uploa...,Açaí,7.99,61,4.96,5300108,10.0,Raruty Açaí Raiz,DNR $$ MPAY $$ MOVPAY_MC $$ MC $$ GPY_ELO $$ E...,CHEAPEST,0.0,ADDRESS_PREFORM_TYPE $$ GUIDED_HELP_TYPE $$ ME...,https://www.ifood.com.br/delivery/brasilia-df/...
2,False,https://static-images.ifood.com.br/image/uploa...,Bebidas,11.99,70,8.35,5300108,5.0,Toma na Kombi,DNR $$ MPAY $$ MOVPAY_MC $$ MC $$ GPY_ELO $$ R...,MODERATE,0.0,ADDRESS_PREFORM_TYPE $$ CPGN_USER_DISCOUNT_6_L...,https://www.ifood.com.br/delivery/brasilia-df/...
3,False,https://static-images.ifood.com.br/image/uploa...,Carnes,16.49,63,6.35,5300108,20.0,Churrasquinho do Barriga´s,DNR $$ MPAY $$ MOVPAY_MC $$ MC $$ GPY_ELO $$ E...,CHEAPEST,0.0,ADDRESS_PREFORM_TYPE $$ GUIDED_HELP_TYPE $$ NO...,https://www.ifood.com.br/delivery/brasilia-df/...
4,False,https://static-images.ifood.com.br/image/uploa...,Brasileira,11.99,58,5.22,5300108,20.0,Prime Restaurante,DNR $$ MPAY $$ MOVPAY_MC $$ MC $$ GPY_ELO $$ E...,CHEAPEST,0.0,ADDRESS_PREFORM_TYPE $$ GUIDED_HELP_TYPE $$ NOVO,https://www.ifood.com.br/delivery/brasilia-df/...


In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

totalAvList = []
nav = webdriver.Chrome(ChromeDriverManager().install())

for i in range(0, len(noData['url'])):
    url = noData['url'][i]

    nav.get(url)

    ##se o site ao estiver inativo
    if not nav.title == 'Delivery de Comida e Mercado - iFood':
        try:
            nav.find_element(By.CLASS_NAME,'restaurant-rating__rating-wrapper').click()
            # content.click()

            content = nav.find_element(By.CLASS_NAME, 'rating-counter__total')

            totalAv = content.text
            totalAv = totalAv.split()
            totalAvList.append(totalAv[0])
            # print(totalAv[0])
        except:
            totalAvList.append(0)
            

    else:
        totalAvList.append(0)

nav.quit()

# url = noData['url'][0]

# nav = webdriver.Chrome(ChromeDriverManager().install())
# nav.get(url)

# ##se o site ao estiver inativo
# if not nav.title == 'Delivery de Comida e Mercado - iFood':
#     nav.find_element(By.CLASS_NAME,'restaurant-rating__rating-wrapper').click()
#     # content.click()

#     content = nav.find_element(By.CLASS_NAME, 'rating-counter__total')

#     totalAv = content.text
#     totalAv = totalAv.split()
#     totalAvList.append(totalAv[0])
#     print(totalAv[0])


# nav.quit()

# print(content.text)

/tmp/ipykernel_68181/1896117948.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  nav = webdriver.Chrome(ChromeDriverManager().install())


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.124)
Stacktrace:
#0 0x55f63b8962a3 <unknown>
#1 0x55f63b654f77 <unknown>
#2 0x55f63b630515 <unknown>
#3 0x55f63b6b5fff <unknown>
#4 0x55f63b6c8e43 <unknown>
#5 0x55f63b6b1903 <unknown>
#6 0x55f63b684ece <unknown>
#7 0x55f63b685fde <unknown>
#8 0x55f63b8e663e <unknown>
#9 0x55f63b8e9b79 <unknown>
#10 0x55f63b8cc89e <unknown>
#11 0x55f63b8eaa83 <unknown>
#12 0x55f63b8bf505 <unknown>
#13 0x55f63b90bca8 <unknown>
#14 0x55f63b90be36 <unknown>
#15 0x55f63b927333 <unknown>
#16 0x7f7560e88609 start_thread


In [11]:
totalAvList

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 '118',
 0,
 0,
 '50',
 0,
 0,
 '2',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 '13',
 '4',
 0,
 0,
 '117',
 '31',
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]